In [1]:
import pandas as pd
import altair as alt
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'

## <font color="#747a7f">6.3</font> Consultas Dinâmicas

Consultas dinâmicas permitem uma exploração mais rápida e também reversível dos dados, útil para separar os padrões de interesse. Conforme definido por [Ahlberg, Williamson e Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), uma consulta dinâmica:

- Representa uma consulta graficamente,
- Fornece limites visíveis para o intervalo da consulta,
- Apresenta uma representação gráfica dos dados e do resultado da consulta,
- Oferece feedback imediato do resultado após cada ajuste da consulta,
- Permite que usuários iniciantes comecem a trabalhar, mesmo com pouco treinamento.

Uma abordagem comum é modificar os parâmetros da consulta usando elementos padrão da interface do usuário, como controles deslizantes, botões de rádio e menus suspensos. Para gerar *widgets* de consulta dinâmica, podemos aplicar a operação `bind` de uma seleção a um ou mais campos de dados que queremos fazer uma consulta.

Vamos construir um gráfico de dispersão interativo que usa uma consulta dinâmica para filtrar a sua exibição. Dado um gráfico de dispersão com classificações de filmes (do *Rotten Tomatoes* e IMDB), podemos adicionar uma seleção no campo `Major_Genre` para permitir uma filtragem com interatividade por gênero de filme.

Para começar, vamos extrair os gêneros únicos (não nulos) dos dados em `movies`:

In [2]:
df = pd.read_json(movies) # Dados dos filmes
genres = df['Major_Genre'].unique() # Valores únicos do campo Major_Genre
genres = list(filter(lambda d: d is not None, genres)) # Filtra valores Nulos/"None"
genres.sort() # Ordem alfabética

Para usarmos depois, vamos também anotar em uma lista os valores em `MPAA_Rating`:

In [3]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Agora, vamos criar uma seleção única associada a um menu suspenso usando a seleção `point`.

*Use o menu de consulta dinâmica abaixo para explorar os dados. Como as avaliações variam por gênero? Como você modificaria o código para filtrar por `MPAA_Rating` (G, PG, PG-13, etc.) em vez de `Major_Genre`?*

In [4]:
selectGenre = alt.selection_point(
    name='Select',  # Nomeia a seleção como 'Select'
    fields=['Major_Genre'],  # Limita a seleção ao campo Major_Genre
    bind=alt.binding_select(options=genres)  # Vincula a um menu com os valores únicos de gênero
)

alt.Chart(movies).mark_circle().add_params(
    selectGenre  # Adiciona a seleção ao gráfico
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))  # Ajusta a opacidade com base na seleção
)

alt.Chart(...)

Nossa construção acima aproveita vários aspectos das seleções:

- Damos um nome à seleção (`'Select'`). Esse nome não é obrigatório, mas nos permite modificar o texto no rótulo do menu de consulta dinâmica gerado. (*O que acontece se você remover o nome? Teste!*)
- Restringimos a seleção a um campo de dados específico (`Major_Genre`). Anteriormente, quando usamos uma seleção única pontual, ela estava associada a pontos de dados individuais. Ao limitar a seleção a um campo específico, podemos selecionar *todos* os pontos de dados cujo valor no campo Major_Genre corresponda ao valor selecionado.
- Inicializamos a seleção com um valor inicial usando `init=....`
- Vinculamos com o `bind` a seleção a um elemento da interface, neste caso, um menu suspenso usando `binding_select`.
- Como antes, usamos ainda uma codificação condicional para controlar o canal de opacidade.

### <font color="#747a7f">6.3.1</font> Vinculando Seleções à Múltiplas Entradas

Uma única instância de seleção pode ser vinculada a vários widgets de consulta dinâmica. Vamos modificar o exemplo acima para fornecer filtros para ambos, `Major_Genre` e `MPAA_Rating`, usando botões de rádio em vez de um menu suspenso. Nossa seleção única agora é definida por um único *par* de valores de gênero e classificação MPAA.

*Procure combinações inesperadas de gênero e classificação. Existem filmes de terror classificados como G ou PG?*

In [8]:
# Seleção de valor único sobre pares [Major_Genre, MPAA_Rating]
# Usa valores específicos predefinidos como valores iniciais
selection = alt.selection_point(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    bind={
        'Major_Genre': alt.binding_select(options=genres),
        'MPAA_Rating': alt.binding_radio(options=mpaa)
    }
)

# Gráfico de dispersão, modifica a opacidade com base na seleção
alt.Chart(movies).mark_circle().add_params( 
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)


alt.Chart(...)

*Curiosidades: A classificação PG-13 não existia quando os filmes Tubarão e Tubarão 2 foram lançados. O primeiro filme a receber a classificação PG-13 foi Amanhecer Violento, de 1984.*

### <font color="#747a7f">6.3.2</font> Usando Visualizações como Consultas Dinâmicas

Embora os *widgets* padrão da interface mostrem os possíveis valores dos parâmetros da consulta, eles não visualizam a distribuição desses valores. Também podemos quere implementar uma interatividade mais rica, como seleções múltiplas ou intervalos, em vez de widgets de entrada que selecionam apenas um valor por vez.

Para resolver esses problemas, podemos criar gráficos adicionais para visualizar os dados e fornecer suporta para as consultas dinâmicas. Vamos adicionar um histograma da contagem de filmes por ano e usar uma seleção de intervalo para destacar dinamicamente os filmes em períodos de tempo selecionados.

*Interaja com o histograma de anos para explorar filmes de diferentes períodos. Você vê algum indício de viés de amostragem ao longo dos anos? (Como os anos e as classificações dos críticos se relacionam?)*

*Os anos variam de 1930 a 2040! Os filmes futuros estão em pré-produção, ou há erros do tipo "erro por um século"? Além disso, dependendo de qual fuso horário você está, pode ver um pequeno pico em 1969 ou 1970. Por que isso acontece? (Veja o final do notebook para uma explicação!)*

In [10]:
brush = alt.selection_interval(
    encodings=['x']  # Limita a seleção aos valores do eixo x (ano)
)

# histograma de consulta dinâmica
years = alt.Chart(movies).mark_bar().encode(
    alt.X('year(Release_Date):T', title='Filmes por Ano de Lançamento'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
).add_params(brush)  # Adiciona a seleção ao gráfico de anos

# gráfico de dispersão, modifica a opacidade com base na seleção
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)


alt.VConcatChart(...)

O exemplo acima fornece consultas dinâmicas usando uma *seleção vinculada* entre gráficos:

- Criamos uma *seleção de intervalo* (`brush`) e definimos `encodings=['x']` para limitar a seleção apenas ao eixo x, resultando em um intervalo de seleção unidimensional.
- Registramos o `brush` com nosso histograma de filmes por ano meio de `.add_params(brush)`.
- Usamos o `brush` em uma codificação condicional para ajustar a opacidade do gráfico de dispersão com `opacity`.
- Essa técnica de interação de selecionar elementos em um gráfico e ver destaques vinculados em um ou mais outros gráficos é conhecida como [brushing & linking]() (ou seleção e vínculo).

## <font color="#747a7f">6.4</font> Zoom e Deslocamento

O gráfico de dispersão das classificações dos filmes está um pouco congestionado em alguns pontos, o que dificulta a análise das regiões mais densas. Usando as técnicas de interação de deslocamento (*panning*) e *zoom*, podemos inspecionar as regiões densas de forma mais detalhada.

Vamos começar pensando em como poderíamos expressar o deslocamento e zoom usando seleções do Altair. O que define a "área de visualização" de um gráfico? *Os domínios das escalas dos eixos!*

Podemos alterar os domínios das escalas para modificar o intervalo visualizado dos valores dos dados. Para fazer isso de forma interativa, podemos vincular uma *seleção de intervalo* aos domínios das escalas com o código `bind='scales'`. O resultado é que, em vez de "*brush*" no intervalo que podemos arrastar e fazer zoom, podemos arrastar e fazer zoom em toda a área do gráfico!

*No gráfico abaixo, clique e arraste para mover a visualização ou use a roda do mouse para fazer zoom (ajustar a escala) na visualização. O que você pode descobrir sobre a precisão dos valores de classificação fornecidos?*

In [11]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)),  # Usa minExtent para estabilizar o título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).add_params(
    alt.selection_interval(bind='scales')  # Atualiza para usar add_params
).properties(
    width=600,
    height=400
)

alt.Chart(...)

*Ao fazer zoom, podemos ver que os valores das classificações têm precisão limitada! As classificações do Rotten Tomatoes são inteiros, enquanto as classificações do IMDB são truncadas para décimos. Como resultado, há sobreposição de pontos mesmo quando fazemos zoom, com vários filmes compartilhando os mesmos valores de classificação.*

Lendo o código acima, você pode notar o código `alt.Axis(minExtent=30)` no canal de codificação do eixo y. O parâmetro `minExtent` garante que um espaço mínimo seja reservado para os marcadores e rótulos dos eixos. Por que fazer isso? Quando fazemos o deslocamento e o zoom, os rótulos dos eixos podem mudar e causar um deslocamento na posição do título do eixo. Ao definir um valor mínimo de extensão, podemos reduzir movimentos que podem causar distração. *Tente alterar o valor de `minExtent`, por exemplo, definindo-o como zero, e depois faça o zoom para ver o que acontece quando rótulos de eixos mais longos entram na visualização.*

O Altair também inclui uma forma abreviada para adicionar deslocamento e zoom a um gráfico. Em vez de criar diretamente uma seleção, você pode chamar a função `.interactive()` para fazer o Altair gerar automaticamente uma seleção de intervalo vinculada às escalas do gráfico.

In [12]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use min extent to stabilize axis title placement
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

alt.Chart(...)

Por padrão, as vinculações de escala para seleções incluem os canais de codificação tanto no eixo x quanto no eixo y. E se quisermos limitar o deslocamento e o zoom a uma única dimensão? Podemos usar `encodings=['x']` para restringir a seleção apenas ao canal `x`:

In [ ]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)),  # Usa minExtent para estabilizar o título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).add_params(
    alt.selection_interval(bind='scales', encodings=['x'])  # Usando add_params em vez de add_selection
).properties(
    width=600,
    height=400
)

alt.Chart(...)

*Ao fazer zoom ao longo de um único eixo, a forma dos dados visualizados pode mudar, afetando potencialmente nossa percepção das relações nos dados. [Escolher uma proporção adequada](http://vis.stanford.edu/papers/arclength-banking) é uma consideração importante no design de visualizações!*